In [1]:
import pandas as pd
# from tabulate import tabulate
import numpy as np
from pygobnilp.gobnilp import Gobnilp
# import bnlearn as bn
import pgmpy
import itertools
from scipy.stats import chi2_contingency #check for independence between 2 variables
from scipy.stats import power_divergence
from pgmpy.estimators.CITests import log_likelihood
from pgmpy.estimators.CITests import chi_square
# from pgmpy.factors.continuous.discretize import BaseDiscretizer

In [2]:
def csv_format_discrete(csv_file):
    df = pd.read_csv(csv_file, sep="\s+")
    return df.drop([0])

#returns the csv_file in a pandas dataframe, formatted properly, discrete dataset only

In [3]:
df = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\alarm_100.dat")
df
df_test = csv_format_discrete(r"C:\Users\User\Documents\GitHub\ML_FYP\dataset\alarm_10000.dat")
df_test

,PULMEMBOLUS,PAP,KINKEDTUBE,INTUBATION,MINVOLSET,VENTMACH,DISCONNECT,VENTTUBE,VENTLUNG,SHUNT,...,CO,BP,EXPCO2,ERRLOWOUTPUT,PRESS,HRBP,MINVOL,HISTORY,HRSAT,PCWP
1,1,1,0,1,2,2,1,3,3,0,...,1,1,1,0,0,0,3,0,0,2
2,0,2,0,1,2,2,0,1,3,1,...,0,0,1,1,1,2,3,0,0,2
3,0,2,0,2,2,2,0,1,1,0,...,0,0,2,0,0,0,1,0,0,2
4,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,0,0,3,1,2,1
5,0,2,0,1,2,2,0,1,3,1,...,0,0,1,0,1,1,3,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,0,2,1,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0
9997,0,0,0,1,2,2,0,1,3,1,...,0,0,1,0,2,0,3,0,0,2
9998,0,2,0,0,0,0,0,0,2,1,...,0,0,2,0,0,0,3,0,0,2
9999,0,2,0,1,2,2,0,1,3,1,...,1,1,1,0,1,0,3,0,0,0


### chi-squared test

In [4]:
#0th order chi2 test
def chi2bool(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=True) #returns chi, p_value, dof
        empty_4.append([j, i , chisquare])
        
    return empty_4

In [5]:
def chi2val(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        chisquare = chi_square(X=x, Y=y, Z=[], data=df, significance_level=0.05, boolean=False) #returns chi, p_value, dof
        empty_4.append([j, i , chisquare])
        
    return empty_4


### log-likelihood tests

In [6]:
def g2val(df):
#returns a tuple(chi2, p_value, dof) if boolean = false
#the null hypothesis is that they are independent of each other
#if true, the p_value is higher than the significance test, we do not reject the null hypothesis
#if false, the p_value is lower than the significance test, we reject the null hypothesis
    v = list(df)
    empty = []
    empty_1 = []
    empty_2 = []
    empty_3 = []
    empty_4 = []
    itertools_combinations = list(itertools.combinations(v, 2)) #finds every possible combination of list(df)
    for i in itertools_combinations:
        empty_1.append(i[0])
        empty_2.append(i[1])
        
    total_number_of_combinations = len(empty_1)
    
    for t in range(len(itertools_combinations)):
        empty.append(itertools_combinations[t])
        
    for value in range(len(empty_1)):
        empty_3.append(value)
        
    for x,y,i,j in zip(empty_1, empty_2, empty, empty_3):
        g2 = log_likelihood(X=x, Y=y, Z=[], significance_level=0.05, boolean=True, data=df) 
        empty_4.append([j, i , g2])
        
    return empty_4

### Sorting Functions

In [7]:
def removeDuplicates(lst):
    return [t for t in (set(tuple(i) for i in lst))]

In [8]:
def sortReturn(data):
    #just a sorting function
    true_list = []
    false_list = []
    for i in data:
        if True in i:
            true_list.append(i)
        if False in i:
            false_list.append(i)
    return true_list, false_list

In [9]:
#returns a list with a tuple of combinations of false
def false2tuple(data):
    empty = []
    false_list_of_tuples = []
    new_df = pd.DataFrame(data)
    newer_df = new_df[1]
    return newer_df

In [10]:
#given 2 lists, append them together and remove duplicates
def remove_dupes(list_1, list_2):
    for i in list_2:
        list_1.append(i)
    remove_dupes = list(set(list_1))
    return remove_dupes

### CONDITIONAL MAIN CODE

In [11]:
#returns TRUE/FALSE Xs and Ys in a tuple of (X,Y)
def conditional_sort(df):
    output_false = []
    output_true = []
    my_list = [x for x in chi2bool(df) if False in x]
    my_list_2 = [x for x in chi2bool(df) if True in x]
    for i,j in zip(my_list, my_list_2):
        output_true.append(j[1])
        output_false.append(i[1])
    return output_true, output_false

In [12]:
#returns permutations of x,y,z where x,y does not repeat e.g (a,b,c), (b,a,c)
def conditional_permute(df):
    a = conditional_sort(df)[1] #obtains the list of false outputs, false output means dependent
    k = list(itertools.permutations(df,3))
    permute_list = []
    for i in a:
        permute = list(itertools.permutations(i))
        permute_list.append(permute[1]) #returns a list of permuted items from a
    for items in permute_list:
        a.append(items) #adds all possible permutations to a 
    my_list = [x for x in k if all(x[:2] != y[:2] for y in a)] #checks if all elements in x[:2] != y[:2]
    my_list_2 = [x[:2] for x in my_list]
    a0 = list(tuple(sorted(l)) for l in my_list_2)
    output = [x for x in my_list if x[:2] in a0] #the three lines fixes all the permutations 
    return output

In [13]:
def conditional_1st_order(df):
    b0 = conditional_permute(df)
    b_100 = []
    for i,j,k in b0:
        chi2 = chi_square(X=i, Y=j, Z=[k], data=df, significance_level=0.05)
        b_100.append((i,j,k, chi2))
    b_100_true = [x for x in b_100 if True in x]
    b_100_false = [x for x in b_100 if False in x]
    return b_100_true, b_100_false

In [53]:
#a function to merge and remove permutations something
def merger(df, n):
    #0th order
    cond_0 = chi2bool(df)
    true_list_0 = [x[1] for x in cond_0 if True in x]
    #0th order
    #1st order
    cond_1 = conditional_1st_order(df)
    true_list_1 = [x[:2] for x in cond_1[0]]
    #1st order
    #true_list
    true_list = true_list_0 + true_list_1
    #true_list
    chi2_ind = list(set(tuple(sorted(l)) for l in true_list))
    
    chi2_permute = []
    for i in chi2_ind:
        delta = list(itertools.permutations(i, 2))
        chi2_permute.append(delta[1])
    
    chi2_permutations = chi2_ind + chi2_permute #0th and 1st order + their permutations
    
    v = list(df)
    k = list(itertools.permutations(v, n)) #nP3 
    t = [x for x in k if x[:2] not in chi_permutations]
    
    t_1 = list(set(tuple(sorted(l[:2])) for l in t))
    
    t_final = [x for x in t if x[:2] not in t_1]
    
    return t_final, chi2_permutations

In [54]:
%%time
k = merger(df, 3)

CPU times: total: 1min 45s
Wall time: 1min 45s


In [59]:
len(k[1])

1040

In [45]:
len(list(df))

37

In [49]:
# def PC_algorithm(n, df):
k

()

In [47]:
# %%time
# N = 2
# testing_list = []
# my_list = [x for x in itertools.permutations(v, N) if x[:2] not in chi2_ind]
# m_1 = [x[0] for x in my_list]
# m_2 = [x[1] for x in my_list]
# m_3 = [x[2:] for x in my_list]
# #         print(i[0], i[1], i[2:N])
# for i,j,k in zip(m_1, m_2, m_3):
#     phi = chi_square(X=i, Y=j, Z=k, data=df_test, significance_level=0.05)
#     testing_list.append([i,j,k,phi])
# testing_list

In [66]:
remove_list = [x for x in k[1]]

In [88]:
%%time
N = 4
n = 4
v = list(df)
while N <= n:
    empty_list = []
    list_permutations = [x for x in itertools.permutations(v, N) if x[:2] not in remove_list]
    p_1 = [x[:1] for x in list_permutations]
    p_2 = [x[:2] for x in list_permutations]
    p = [x[2:] for x in list_permutations]
    for i,j,k in zip(p_1, p_2, p):
        chi_square(X=i, Y=j, Z=k, data=df, significance_level=0.95)
        empty_list.append((i,j,k,chi_square))
    true_list = [x for x in empty_list if True in x]
    false_list = [x for x in empty_list if False in x]
    N += 1

KeyError: ('One',)

In [93]:
%%time
empty_list = []
list_permutations = [x for x in itertools.permutations(v, N) if x[:2] not in remove_list]
p_1 = [x[:1] for x in list_permutations]
p_2 = [x[:2] for x in list_permutations]
p = [x[2:] for x in list_permutations]
#     empty_list.append((i,j,k,chi_square))
# true_list = [x for x in empty_list if True in x]
# false_list = [x for x in empty_list if False in x]

CPU times: total: 27.7 s
Wall time: 28 s


In [ ]:
for i,j,k in zip(p_1, p_2, p):
    chi_square(X=i, Y=j, Z=[k], data=df, significance_level=0.95)

In [84]:
list_permutations

[('One', 'Two', 'Three', 'Four'),
 ('One', 'Two', 'Three', 'Five'),
 ('One', 'Two', 'Three', 'Six'),
 ('One', 'Two', 'Three', 'Seven'),
 ('One', 'Two', 'Three', 'Eight'),
 ('One', 'Two', 'Three', 'Nine'),
 ('One', 'Two', 'Three', 'Ten'),
 ('One', 'Two', 'Three', 'Eleven'),
 ('One', 'Two', 'Three', 'Twelve'),
 ('One', 'Two', 'Three', 'Thirteen'),
 ('One', 'Two', 'Three', 'Fourteen'),
 ('One', 'Two', 'Three', 'Fifteen'),
 ('One', 'Two', 'Three', 'Sixteen'),
 ('One', 'Two', 'Three', 'Seventeen'),
 ('One', 'Two', 'Three', 'Eighteen'),
 ('One', 'Two', 'Three', 'Nineteen'),
 ('One', 'Two', 'Three', 'Twenty'),
 ('One', 'Two', 'Three', 'TwentyOne'),
 ('One', 'Two', 'Three', 'TwentyTwo'),
 ('One', 'Two', 'Three', 'TwentyThree'),
 ('One', 'Two', 'Three', 'TwentyFour'),
 ('One', 'Two', 'Three', 'TwentyFive'),
 ('One', 'Two', 'Three', 'TwentySix'),
 ('One', 'Two', 'Three', 'TwentySeven'),
 ('One', 'Two', 'Three', 'TwentyEight'),
 ('One', 'Two', 'Three', 'TwentyNine'),
 ('One', 'Two', 'Three', 'Thi

In [80]:
%%time
p = [x for x in list_permutations if x[:2] not in remove_list]
len(p)

CPU times: total: 9.44 s
Wall time: 9.47 s


347480